In [12]:
%matplotlib inline
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, \
    MaxPooling2D, \
    Convolution2D, \
    Dropout, \
    BatchNormalization, \
    ZeroPadding2D, \
    Lambda, \
    Flatten
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.datasets import mnist
import matplotlib.pyplot as plt
from utils import save_array, \
    load_array

## Simple Linear Model

### Create Dummy Dataset

In [2]:
n = 1000
p = 3
X = np.random.rand(n, p)
w = np.array([5, 7, 2])
y = np.dot(X, w)

### Fit Model

In [46]:
model = Sequential([Dense(1, input_dim=3)])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(X, y, nb_epoch=100)

Epoch 1/100
1000/1000 [==============================] - 0s - loss: 25.1412     
Epoch 2/100
1000/1000 [==============================] - 0s - loss: 4.4256     
Epoch 3/100
1000/1000 [==============================] - 0s - loss: 2.2725     
Epoch 4/100
1000/1000 [==============================] - 0s - loss: 1.9321     
Epoch 5/100
1000/1000 [==============================] - 0s - loss: 1.7843     
Epoch 6/100
1000/1000 [==============================] - 0s - loss: 1.6588     
Epoch 7/100
1000/1000 [==============================] - 0s - loss: 1.5427     
Epoch 8/100
1000/1000 [==============================] - 0s - loss: 1.4380     
Epoch 9/100
1000/1000 [==============================] - 0s - loss: 1.3400     
Epoch 10/100
1000/1000 [==============================] - 0s - loss: 1.2498     
Epoch 11/100
1000/1000 [==============================] - 0s - loss: 1.1663     
Epoch 12/100
1000/1000 [==============================] - 0s - loss: 1.0891     
Epoch 13/100
1000/1000 [============

In [47]:
model.layers[0].get_weights()

[array([[ 4.87689924],
        [ 6.86038113],
        [ 1.86813855]], dtype=float32), array([ 0.20528407], dtype=float32)]

## DogsVsCats Linear Model using VGG outputs

In [3]:
def onehot_keras(x):
    return to_categorical(x)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
y_train = onehot_keras(y_train)
y_test = onehot_keras(y_test)
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))
model_url = 'http://files.fast.ai/models/vgg16.h5'
model_save_folder = '/mnt/models/mnist/'
model_file_name = 'conv.h5'
train_preds_file_name = 'train_preds.bc'
test_preds_file_name = 'test_preds.bc'

In [5]:
def get_batches(path, batch_size=4, idg=image.ImageDataGenerator(), shuffle=True, class_mode='categorical'):
    return idg.flow_from_directory(path, batch_size=batch_size, target_size=(224, 224), 
                                   shuffle=shuffle, class_mode=class_mode)

In [6]:
def addConvBlock(model, layers, filters):
    for layer in range(layers):
        model.add(ZeroPadding2D(padding=(1, 1)))
        model.add(Convolution2D(nb_filter=filters, nb_col=3, nb_row=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

In [7]:
def addFCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [8]:
X_mean = np.mean(X_train).astype('float32')
X_sd = np.std(X_train).astype('float32')
def pre_process(X):
    X = (X - X_mean)/X_sd
    return X

In [9]:
def create():
    model = Sequential()
    model.add(Lambda(pre_process, input_shape=(1, 28, 28), output_shape=(1, 28, 28)))
    addConvBlock(model, 2, 64)
    addConvBlock(model, 2, 128)
    model.add(Flatten())
    addFCBlock(model)
    addFCBlock(model)
    model.add(Dense(10, activation='softmax'))
    return model

In [10]:
model = create()

In [44]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_6 (Lambda)                (None, 1, 28, 28)     0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
zeropadding2d_48 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_6[0][0]                   
____________________________________________________________________________________________________
convolution2d_48 (Convolution2D) (None, 64, 28, 28)    640         zeropadding2d_48[0][0]           
____________________________________________________________________________________________________
zeropadding2d_49 (ZeroPadding2D) (None, 64, 30, 30)    0           convolution2d_48[0][0]           
___________________________________________________________________________________________

In [13]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, nb_epoch=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 7590s - loss: 0.1432 - acc: 0.9568 - val_loss: 0.0418 - val_acc: 0.9877


In [17]:
model.save_weights(model_save_folder + model_file_name)

In [ ]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

In [13]:
# save_array(model_save_folder + train_preds_file_name, train_preds)
# save_array(model_save_folder + test_preds_file_name, test_preds)
train_preds = load_array(model_save_folder + train_preds_file_name)
test_preds = load_array(model_save_folder + test_preds_file_name)

### Build linear model using predictions

In [14]:
lm_model = Sequential([Dense(10, activation='softmax', input_dim=10)])

In [15]:
lm_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_4 (Dense)                  (None, 10)            110         dense_input_1[0][0]              
Total params: 110
Trainable params: 110
Non-trainable params: 0
____________________________________________________________________________________________________


In [16]:
lm_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
lm_model.fit(train_preds, y_train, validation_data=(test_preds, y_test), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 7s - loss: 1.2470 - acc: 0.8488 - val_loss: 0.5132 - val_acc: 0.9874
